In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from ir_measures import RR, R, nDCG, read_trec_run
import ir_datasets
import json
import gzip

In [18]:
df = []
for ncells in [1, 2, 4, 8]:
  for centroid_score_threshold in [0.4, 0.45, 0.5, 0.6, 0.75]:
    for ndocs in [256, 1024, 4096, 8192]:
        id = f'plaid.ncells-{ncells}.cst-{centroid_score_threshold}.ndocs-{ndocs}'
        latency = float(open(f'results/devs.{id}.time').read())
        res_dl19 = json.load(gzip.open(f'results/dl19.{id}.run.gz.measures.json.gz'))
        res_devs = json.load(gzip.open(f'results/devs.{id}.run.gz.measures.json.gz'))
        df.append({
            'ncells': ncells,
            'cst': centroid_score_threshold,
            'ndocs': ndocs,
            'devs_latency': latency,
            'dl19_ndcg10': res_dl19['avg']['nDCG@10'],
            'dl19_ndcg1k': res_dl19['avg']['nDCG@1000'],
            'dl19_r1k': res_dl19['avg']['R(rel=2)@1000'],
            'dl19_rbo99': res_dl19['avg']['RBO(p=0.99)'],
            'devs_rr10': res_devs['avg']['RR@10'],
            'devs_r1k': res_devs['avg']['R@1000'],
            'devs_rbo99': res_devs['avg']['RBO(p=0.99)'],
        })
res_dl19 = json.load(gzip.open(f'results/dl19.exhuastive.run.gz.measures.json.gz'))
res_devs = json.load(gzip.open(f'results/devs.exhuastive.run.gz.measures.json.gz'))
df.append({
    'ncells': 0,
    'cst': 0,
    'ndocs': 0,
    'devs_latency': None,
    'dl19_ndcg10': res_dl19['avg']['nDCG@10'],
    'dl19_ndcg1k': res_dl19['avg']['nDCG@1000'],
    'dl19_r1k': res_dl19['avg']['R(rel=2)@1000'],
    'dl19_rbo99': res_dl19['avg']['RBO(p=0.99)'],
    'devs_rr10': res_devs['avg']['RR@10'],
    'devs_r1k': res_devs['avg']['R@1000'],
    'devs_rbo99': res_devs['avg']['RBO(p=0.99)'],
})
df = pd.DataFrame(df)
df = df[df.ncells!=100]

In [19]:
measure = 'devs_rbo99'

for i, (ncells, cst, ndocs) in enumerate([(1, .5, 256), (2, .45, 1024), (4, .4, 4096), (0,0,0)]):
    subdf = df[(df.ncells==ncells) & (df.cst==cst) & (df.ndocs==ndocs)]
    assert len(subdf) == 1
    r = subdf.iloc[0]
    print(f'& {r.devs_rr10:.3f} & {r.devs_r1k:.3f} & {r.devs_rbo99:.3f} & {r.devs_latency:.1f} & {r.dl19_ndcg10:.3f} & {r.dl19_ndcg1k:.3f} & {r.dl19_r1k:.3f} \\\\')
    

& 0.394 & 0.833 & 0.612 & 80.5 & 0.739 & 0.553 & 0.555 \\
& 0.397 & 0.933 & 0.890 & 103.4 & 0.745 & 0.707 & 0.786 \\
& 0.397 & 0.975 & 0.983 & 163.9 & 0.745 & 0.760 & 0.871 \\
& 0.397 & 0.984 & 1.000 & nan & 0.745 & 0.769 & 0.894 \\
